In [19]:
import pandas as pd
import haversine
import json
import numpy as np

In [20]:
zip_by_store=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/sales_by_zip_by_store_0330.csv",dtype=str)
zip_by_store=zip_by_store[['location_id','zip','revenue_flag']]

In [21]:
zips_1C_2C=pd.read_excel("/home/jian/Projects/Big_Lots/Newspaper/Test_Zips/Store_List_in_1C_2C.xlsx",dtype=str)

In [22]:
store_list_1=pd.read_table("/home/jian/BiglotsCode/OtherInput/MediaStormStoreList_Nov15.txt",dtype=str,sep="|")
store_list_1.head(2)

,location_id,location_desc,open_dt,address_line_1,address_line_2,city_nm,state_nm,zip_cd,longitude_meas,latitude_meas
0,3,PEACH ORCHARD PLAZA,1988-10-31,2708 PEACH ORCHARD RD,NaN,AUGUSTA,GA,30906-2408,-000000000082.018921,000000000033.421570
1,30,SOUTHWOOD PLZ S/C,1984-04-05,818 S MAIN ST,NaN,BOWLING GREEN,OH,43402-4601,-000000000083.650909,000000000041.363132


In [23]:
store_list_1=pd.read_table("/home/jian/BiglotsCode/OtherInput/MediaStormStoreList_Nov15.txt",dtype=str,sep="|")
store_list_2=pd.read_table("/home/jian/BiglotsCode/OtherInput/MediaStormStores_20180703.txt",dtype=str,sep="|")
store_list_1['store_zip']=store_list_1['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
store_list_2['store_zip']=store_list_2['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))

store_list_1=store_list_1[['location_id','store_zip','longitude_meas','latitude_meas']]
store_list_2=store_list_2[['location_id','store_zip','longitude_meas','latitude_meas']]
store_list_1=store_list_1[~store_list_1['location_id'].isin(store_list_2['location_id'])]
store_list=store_list_2.append(store_list_1)
store_list=store_list[store_list['location_id']!="6990"]
store_list=store_list[store_list['location_id']!="145"]


store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))

In [24]:
df_1C=store_list[store_list['location_id'].isin(zips_1C_2C['1C'])]
df_2C=store_list[store_list['location_id'].isin(zips_1C_2C['2C'])]

In [25]:
def merge_zip_selection(df):
    df=pd.merge(df,zip_by_store,on="location_id",how="left")
    df=df[df['revenue_flag'].isin(["P","S"])]
    df['distance']=np.nan
    df=df.reset_index()
    del df['index']
    for i in range(len(df)):
        store_loc=(df['latitude_meas'][i],df['longitude_meas'][i])
        try:
            df['distance'][i]=round(haversine.haversine(store_loc,zip_centers[df['zip'][i]],miles=True),4)
        except:
            df['distance'][i]="NA"
            print(df['zip'][i])
            
        
    return df

In [26]:
df_1C_output=merge_zip_selection(df_1C)
df_1C_output=df_1C_output.sort_values(['location_id','revenue_flag'])
del df_1C_output['longitude_meas']
del df_1C_output['latitude_meas']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


70707
70879
59022
85553
85554


In [27]:
df_2C_output=merge_zip_selection(df_2C)
df_2C_output=df_2C_output.sort_values(['location_id','revenue_flag'])
del df_2C_output['longitude_meas']
del df_2C_output['latitude_meas']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


22402
61425
86515
87375
87327
86504
87325
86503
87311
87313
86505
87319
86511
87328
86512
87326
87321
86510
85269


In [28]:
df_1C_P=df_1C_output[df_1C_output['revenue_flag']=='P']['zip'].unique().tolist()
df_1C_S=df_1C_output[df_1C_output['revenue_flag']=='S']['zip'].unique().tolist()
df_1C_S=[x for x in df_1C_S if x not in df_1C_P]

df_2C_P=df_2C_output[df_2C_output['revenue_flag']=='P']['zip'].unique().tolist()
df_2C_S=df_2C_output[df_2C_output['revenue_flag']=='S']['zip'].unique().tolist()
df_2C_S=[x for x in df_2C_S if x not in df_2C_P]

In [29]:
if len(df_1C_P)>len(df_1C_S):
    df_1C_S=df_1C_S+[np.nan]*(len(df_1C_P)-len(df_1C_S))
else:
    df_1C_P=df_1C_P+[np.nan]*(len(df_1C_S)-len(df_1C_P))
dedup_1C=pd.DataFrame({"1C_P_zips":df_1C_P,"1C_S_zips":df_1C_S},index=range(max(len(df_1C_P),len(df_1C_S))))

In [30]:
if len(df_2C_P)>len(df_2C_S):
    df_2C_S=df_2C_S+[np.nan]*(len(df_2C_P)-len(df_2C_S))
else:
    df_2C_P=df_2C_P+[np.nan]*(len(df_2C_S)-len(df_2C_P))
dedup_2C=pd.DataFrame({"2C_P_zips":df_2C_P,"2C_S_zips":df_2C_S},index=range(max(len(df_2C_P),len(df_2C_S))))

In [31]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Newspaper/Test_Zips/BL_Targeting_Zips_For_1C_2C_Stores_JL_20180801.xlsx",engine="xlsxwriter")
dedup_1C.to_excel(writer,"dedup_1C",index=False)
dedup_2C.to_excel(writer,"dedup_2C",index=False)
df_1C_output.to_excel(writer,"1C",index=False)
df_2C_output.to_excel(writer,"2C",index=False)
writer.save()

In [32]:
dedup_1C.head(2)

,1C_P_zips,1C_S_zips
0,30014,30025
1,30016,31064
